In [1]:
import requests, json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# The beginning and end time of daily observation.
obs_begin = '12'
obs_stop = '20'

# Scrawling the website: __[NEO Earth Close Approaches](https://cneos.jpl.nasa.gov/ca/)__
* Table Settings: Nominal dist. <= __0.2__au

In [3]:
res = requests.get("https://ssd-api.jpl.nasa.gov/cad.api?www=1&nea-comet=Y&dist-max=0.2&fullname=true")
soup = BeautifulSoup(res.text, 'lxml')
# All data for each object.
jd = json.loads(soup.text)

# Choosing NEAs by today's __Close-Approach (CA) Date__

In [4]:
today = datetime.now().strftime("%Y-%b-%d")
neas = []
for neaData in jd['data']:
    if today in neaData[3]:
        neas.append(neaData[0].split('|')[1])

# Scrawling the website: __[Minor Planet & Comet Ephemeris Service](https://www.minorplanetcenter.net/iau/MPEph/MPEph.html)__
## Storing observable NEAs(__neaBright__)' _motions ("/s)_ (__neaMotions__) and _lines of their ephemerides in ACP format_ (__neaLines__).

In [5]:
dateWP = datetime.now().strftime("%Y %m %d")
date = datetime.now().strftime("%Y%m%d")
neaMotions, neaLines, neaBright = [], [], []
for nea in neas:
    neaLine = ""
    payload = {'ty':'e', 'TextArea':nea, 'd':dateWP, 'l':'', 'i':'1', 'u':'h', 'uto':'0', 'c':'D37'
               , 'long':'', 'lat':'', 'alt':'', 'raty':'d', 's':'t', 'm':'s', 'igd':'y', 'ibh':'y', 'adir':'N'
               , 'oed':'', 'e':'-2', 'resoc':'', 'tit':'', 'bu':'', 'ch':'c', 'ce':'f', 'js':'f'}
    res = requests.post("https://cgi.minorplanetcenter.net/cgi-bin/mpeph2.cgi", data = payload)
    soup = BeautifulSoup(res.text, 'lxml')

    ephems = [i for i in soup.select('pre')[0].text.split('\n') if dateWP in i]
    ephemsOK = [i for i in ephems if (int(i.split()[14][1:]) >= 40) and (i.split(' ')[3][:2] >= obs_begin) and (i.split(' ')[3][:2] <= obs_stop)]
    if (ephemsOK != []) and (float(ephemsOK[0].split()[10]) <= 18.):
        neaMotions.append(float(ephemsOK[0].split()[11]))
        for ephem in ephemsOK:
    #         with open("D:/LWTdata/LWT_{}/lulinLWT/others/{}_ephem_{}.txt".format(date, nea, ephemsOK[0].split()[11]), 'a') as file:
    #             file.write(ephem + '\n')
            neaLine = neaLine + ephem[:36] + '|'
        neaLines.append(neaLine[:-1])
        neaBright.append(nea)
    else:
        pass

# If there are some suitable NEAs, choose the one whose observable time spans the longest; otherwise, send an email about no NEO and NEA is suitable.

In [6]:
if neaBright != []:
    neaSpans = [(int(line.split('|')[-1].split()[3][:2])-int(line.split('|')[0].split()[3][:2])) for line in neaLines]
    neaMaxSpanIdx = neaSpans.index(max(neaSpans))
    neaBOK = neaBright[neaMaxSpanIdx]
    neaLOK = neaLines[neaMaxSpanIdx]
    neaMOK = neaMotions[neaMaxSpanIdx]
else:
#     shutil.rmtree("D:/LWTdata/LWT_{}/lulinLWT".format(date))
#     sendemail(from_addr    = 'lwt@gm.astro.ncu.edu.tw', 
#               to_addr_list = ['lwtgroup@astro.ncu.edu.tw'],
#               cc_addr_list = [], 
#               subject      = 'LWT has "NO" observation today!', 
#               message      = 'Bonjour,\n\nPlease run the script of Prof. Ngeow today!\nMerci beaucoup!\n\nAmuse-toi bien,\nJian-Fong Huang (smoBEE)\nemail: smoBEE@astro.ncu.edu.tw', 
#               login        = 'lwt@gm.astro.ncu.edu.tw', 
#               password     = 'lulin1478963')
#     sys.exit()
    pass

In [7]:
print('[{}] motion is: {}"/s'.format(neaBOK, neaMOK))
print('[{}] ephemerise is:\n{}'.format(neaBOK, neaLOK))

[141525] motion is: 0.29"/s
[141525] ephemerise is:
2019 05 01 120000   10.0208  +39.774|2019 05 01 130000   09.9963  +39.709|2019 05 01 140000   09.9719  +39.642|2019 05 01 150000   09.9477  +39.574
